# Práctica: Eliminación Contextual de Stopwords para Análisis de Reseñas de Productos
Este notebook implementa las fases descritas en el archivo `stop-words.md` para analizar y personalizar stopwords en reseñas de productos.

In [3]:
# Fase 1: Análisis Inicial
# Cargar el dataset y procesar texto con spaCy
# %pip install spacy
# !python -m spacy download es_core_news_sm

import pandas as pd
import spacy
from spacy.lang.es.stop_words import STOP_WORDS

# Cargar el dataset
df = pd.read_csv('comentarios_clientes.csv')

# Cargar modelo de spaCy
nlp = spacy.load('es_core_news_sm')

# Función para procesar texto
def procesar_texto(texto):
    doc = nlp(texto)
    return [token.text for token in doc if not token.is_stop]

# Aplicar procesamiento al dataset
df['tokens'] = df['texto'].apply(procesar_texto)

# Mostrar ejemplos
print(df[['texto', 'tokens']].head())

                                                                                                texto  \
El paquete llegó con el embalaje comprometido                aunque lograron reembolsarme rápidamente   
Después de 3 semanas de espera                       el artículo nunca fue despachado. Pésimo segu...   
La atención al cliente fue evasiva y poco resol...                                           negativo   
Entrega express y producto en perfecto estado. ...                                           positivo   
El sistema de seguimiento en línea mostró infor...                                           negativo   

                                                                                               tokens  
El paquete llegó con el embalaje comprometido                [ , lograron, reembolsarme, rápidamente]  
Después de 3 semanas de espera                      [ , artículo, despachado, ., Pésimo, seguimiento]  
La atención al cliente fue evasiva y poco resol...       

In [4]:
# Fase 2: Personalización de la Lista
# Modificar la lista de stopwords
from collections import Counter

# Identificar palabras más frecuentes
todas_palabras = [palabra for tokens in df['tokens'] for palabra in tokens]
frecuencias = Counter(todas_palabras)

# Preservar términos clave
terminos_clave = {'no', 'nunca', 'tampoco', 'pero', 'aunque', 'sin embargo'}

# Eliminar términos genéricos
terminos_genericos = {'producto', 'cliente', 'día', 'hacer', 'tener', 'decir'}

# Añadir stopwords específicas
stopwords_es = STOP_WORDS.copy()
stopwords_es.update({'hola', 'gracias', 'pd'})
stopwords_es.difference_update(terminos_clave)
stopwords_es.update(terminos_genericos)

# Mostrar lista personalizada
print(sorted(stopwords_es))

['a', 'acuerdo', 'adelante', 'ademas', 'además', 'afirmó', 'agregó', 'ahi', 'ahora', 'ahí', 'al', 'algo', 'alguna', 'algunas', 'alguno', 'algunos', 'algún', 'alli', 'allí', 'alrededor', 'ambos', 'ante', 'anterior', 'antes', 'apenas', 'aproximadamente', 'aquel', 'aquella', 'aquellas', 'aquello', 'aquellos', 'aqui', 'aquél', 'aquélla', 'aquéllas', 'aquéllos', 'aquí', 'arriba', 'aseguró', 'asi', 'así', 'atras', 'aun', 'añadió', 'aún', 'bajo', 'bastante', 'bien', 'breve', 'buen', 'buena', 'buenas', 'bueno', 'buenos', 'cada', 'casi', 'cierta', 'ciertas', 'cierto', 'ciertos', 'cinco', 'claro', 'cliente', 'comentó', 'como', 'con', 'conmigo', 'conocer', 'conseguimos', 'conseguir', 'considera', 'consideró', 'consigo', 'consigue', 'consiguen', 'consigues', 'contigo', 'contra', 'creo', 'cual', 'cuales', 'cualquier', 'cuando', 'cuanta', 'cuantas', 'cuanto', 'cuantos', 'cuatro', 'cuenta', 'cuál', 'cuáles', 'cuándo', 'cuánta', 'cuántas', 'cuánto', 'cuántos', 'cómo', 'da', 'dado', 'dan', 'dar', 'de',

In [5]:
# Fase 3: Implementación y Pruebas
# Función para procesar texto con lista personalizada
def procesar_texto_personalizado(texto):
    doc = nlp(texto)
    return [token.text for token in doc if token.text.lower() not in stopwords_es]

# Casos de prueba
casos_prueba = [
    'No funciona bien, pero el diseño es bonito.',
    'Nunca compré algo tan malo. Aunque el precio es bajo, no lo vale.'
    ]

# Probar casos
for caso in casos_prueba:
    print(f'Entrada: {caso}')
    print(f'Salida: {procesar_texto_personalizado(caso)}')
    print('-' * 50)

Entrada: No funciona bien, pero el diseño es bonito.
Salida: ['No', 'funciona', ',', 'pero', 'diseño', 'bonito', '.']
--------------------------------------------------
Entrada: Nunca compré algo tan malo. Aunque el precio es bajo, no lo vale.
Salida: ['Nunca', 'compré', 'malo', '.', 'Aunque', 'precio', ',', 'no', 'vale', '.']
--------------------------------------------------


In [8]:
# Fase 4: Evaluación de Impacto
# Comparar análisis de sentimiento con y sin stopwords personalizadas
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Filtrar filas con valores NaN en 'etiqueta'
df_clean = df.dropna(subset=['etiqueta'])

# Verificar que tenemos datos después del filtrado
print(f'Filas con etiquetas válidas: {len(df_clean)}')

# Vectorizar texto
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_clean['texto'])
y = df_clean['etiqueta']

# Con solo 2 muestras etiquetadas, usamos todo el conjunto para demostración
# En un caso real con más datos, usaríamos stratify=y para mantener la distribución de clases
print(f"Distribución de clases: {y.value_counts().to_dict()}")

# Entrenar modelo básico con todas las muestras
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X, y)

# Evaluar en el mismo conjunto (solo para demostración)
y_pred = modelo.predict(X)
print(f'Exactitud sin stopwords personalizadas: {accuracy_score(y, y_pred)}')

# Vectorizar con stopwords personalizadas
vectorizer_personalizado = CountVectorizer(stop_words=list(stopwords_es))
X_personalizado = vectorizer_personalizado.fit_transform(df_clean['texto'])

# Entrenar modelo con stopwords personalizadas
modelo_p = LogisticRegression(max_iter=1000)
modelo_p.fit(X_personalizado, y)
y_pred_p = modelo_p.predict(X_personalizado)
print(f'Exactitud con stopwords personalizadas: {accuracy_score(y, y_pred_p)}')

Filas con etiquetas válidas: 2
Distribución de clases: {'neutral': 1, 'negativo': 1}
Exactitud sin stopwords personalizadas: 1.0
Exactitud con stopwords personalizadas: 1.0
